# Import Libraries

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import requests

In [4]:
import io

## Get Data

In [40]:
# params = {'select':['location', 'siteid', 'no2', 'nox', 'no', 'pm10', 'pm25'],
#     'where':"siteid = 452 AND date_time IN [date'2022'..date'2022']"}
params = {'select':['location', 'siteid', 'date_time', 'no2', 'nox', 'no', 'pm10', 'pm25'],
    'where':"date_time IN [date'2022'..date'2022']"}


dataset = 'air-quality-data-continuous'

api_url = f"https://opendata.bristol.gov.uk/api/v2/catalog/datasets/{dataset}/exports/csv/"
content = requests.get(api_url, params = params).content

aqdc_df = pd.read_csv(io.StringIO(content.decode('utf-8')), sep = ";")


### Inspect

In [43]:
aqdc_df.head()

,location,siteid,date_time,no2,nox,no,pm10,pm25
0,Parson Street School,215,2022-04-05T11:00:00+00:00,28.063813,49.311689,13.854863,NaN,NaN
1,Temple Way,500,2022-04-05T14:00:00+00:00,13.005000,21.038000,5.239000,23.189,NaN
2,Brislington Depot,203,2022-04-05T14:00:00+00:00,13.015625,23.773969,6.934013,NaN,NaN
3,Temple Way,500,2022-04-05T15:00:00+00:00,12.623000,17.978000,3.492000,19.324,NaN
4,Parson Street School,215,2022-04-05T15:00:00+00:00,22.782951,41.330482,12.090435,NaN,NaN


In [21]:
aqdc_df.shape


(18752, 8)

## Aggregate

In [22]:
aq_summary = aqdc_df.groupby(['location', 'siteid'])[['no2', 'no', 'pm10', 'pm25']].mean().copy()

In [23]:
aq_summary.head(10)

,,no2,no,pm10,pm25
location,siteid,,,,
AURN St Pauls,452,23.402335,14.954005,20.513990,10.720878
Brislington Depot,203,23.543620,17.690243,NaN,NaN
Colston Avenue,501,68.941203,77.996222,NaN,NaN
Fishponds Road,463,31.854295,24.568103,NaN,NaN
Marlborough Street,672,31.979009,26.620450,NaN,NaN
Parson Street School,215,31.245967,30.183083,NaN,20.263735
Temple Way,500,35.597622,31.746998,23.272712,NaN
Wells Road,270,24.506875,27.723810,NaN,NaN


In [44]:
aqdc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18760 entries, 0 to 18759
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   location   18760 non-null  object 
 1   siteid     18760 non-null  int64  
 2   date_time  18760 non-null  object 
 3   no2        18444 non-null  float64
 4   nox        18454 non-null  float64
 5   no         18441 non-null  float64
 6   pm10       4360 non-null   float64
 7   pm25       3134 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 1.1+ MB


In [45]:
aqdc_df['datetime'] = pd.to_datetime(aqdc_df.date_time)

In [46]:
aqdc_df.head()

,location,siteid,date_time,no2,nox,no,pm10,pm25,datetime
0,Parson Street School,215,2022-04-05T11:00:00+00:00,28.063813,49.311689,13.854863,NaN,NaN,2022-04-05 11:00:00+00:00
1,Temple Way,500,2022-04-05T14:00:00+00:00,13.005000,21.038000,5.239000,23.189,NaN,2022-04-05 14:00:00+00:00
2,Brislington Depot,203,2022-04-05T14:00:00+00:00,13.015625,23.773969,6.934013,NaN,NaN,2022-04-05 14:00:00+00:00
3,Temple Way,500,2022-04-05T15:00:00+00:00,12.623000,17.978000,3.492000,19.324,NaN,2022-04-05 15:00:00+00:00
4,Parson Street School,215,2022-04-05T15:00:00+00:00,22.782951,41.330482,12.090435,NaN,NaN,2022-04-05 15:00:00+00:00


 **need to use a mask to subset the columns after the drop operation**

In [47]:
aqdc_df = aqdc_df[aqdc_df.columns.drop('date_time')]

In [48]:
aqdc_df.head()

,location,siteid,no2,nox,no,pm10,pm25,datetime
0,Parson Street School,215,28.063813,49.311689,13.854863,NaN,NaN,2022-04-05 11:00:00+00:00
1,Temple Way,500,13.005000,21.038000,5.239000,23.189,NaN,2022-04-05 14:00:00+00:00
2,Brislington Depot,203,13.015625,23.773969,6.934013,NaN,NaN,2022-04-05 14:00:00+00:00
3,Temple Way,500,12.623000,17.978000,3.492000,19.324,NaN,2022-04-05 15:00:00+00:00
4,Parson Street School,215,22.782951,41.330482,12.090435,NaN,NaN,2022-04-05 15:00:00+00:00


### Now group by a different time period than in the raw data AND some other key(s)

In [56]:
daily_data = aqdc_df.groupby([pd.Grouper(key='datetime', freq='D'), 'location', 'siteid']).mean()

In [57]:
daily_data.head(10)

no2        nox  \
datetime                  location             siteid                         
2022-01-01 00:00:00+00:00 AURN St Pauls        452     11.109687  13.021700   
                          Brislington Depot    203      4.031250  13.927083   
                          Colston Avenue       501     41.687500  94.354167   
                          Fishponds Road       463     12.833333  19.947917   
                          Marlborough Street   672     15.520833  23.750000   
                          Parson Street School 215      9.572917  14.052083   
                          Temple Way           500     28.410971  50.359583   
                          Wells Road           270     12.093750  35.072917   
2022-01-02 00:00:00+00:00 AURN St Pauls        452      6.454712   8.110842   
                          Brislington Depot    203      4.145833  14.447917   

                                                              no       pm10  \
datetime                  location             siteid                         
2022-01-01 00:00:00+00:00 AURN St Pauls        452      1.246987  29.187250   
                          Brislington Depot    203      6.427083        NaN   
                          Colston Avenue       501     34.260417        NaN   
                          Fishponds Road       463      4.635417        NaN   
                          Marlborough Street   672      5.291667        NaN   
                          Parson Street School 215      2.833333        NaN   
                          Temple Way           500     14.314508  23.526958   
                          Wells Road           270     14.989583        NaN   
2022-01-02 00:00:00+00:00 AURN St Pauls        452      1.080088  15.902083   
                          Brislington Depot    203      6.708333        NaN   

                                                            pm25  
datetime                  location             siteid             
2022-01-01 00:00:00+00:00 AURN St Pauls        452     13.416667  
                          Brislington Depot    203           NaN  
                          Colston Avenue       501           NaN  
                          Fishponds Road       463           NaN  
                          Marlborough Street   672           NaN  
                          Parson Street School 215     13.552615  
                          Temple Way           500           NaN  
                          Wells Road           270           NaN  
2022-01-02 00:00:00+00:00 AURN St Pauls        452      4.333333  
                          Brislington Depot    203           NaN